In [ ]:
# 입력 매개변수
# stnIds : 지역 (108 # 서울)
# startDt : 날짜 범위 시작 (19870101)

def get_data(startDt_, stnIds_):
  from urllib.request import Request, urlopen
  from urllib.parse import urlencode, quote_plus
  import bs4
  import pandas as pd
  import numpy as np
  from datetime import timedelta

  url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
  my_key = 'BPRdYP7eFMFJt+UFCA0ghjAgXCfrhXLNczu+kjlxEHqKPrgLiaYG7sUavuCm/tqxC+nJd5spaBjwEDbo/QelXg=='
  
  # 날짜 범위의 최대값을 '20210501'로 설정
  queryParams = '?' + urlencode({ quote_plus('ServiceKey') : my_key, quote_plus('pageNo') : '1', quote_plus('numOfRows') : '999', quote_plus('dataType') : 'XML', quote_plus('dataCd') : 'ASOS', quote_plus('dateCd') : 'DAY', quote_plus('startDt') : startDt_ , quote_plus('endDt') : '20220111', quote_plus('stnIds') : stnIds_ })

  request = Request(url + queryParams)
  request.get_method = lambda: 'GET'
  response_body = urlopen(request).read()

  xmlobj = bs4.BeautifulSoup(response_body, 'lxml-xml')
  items = xmlobj.findAll('item')

  ## 원하는 형태로 가공
  asos = pd.DataFrame()
  for i in items:
    dic_ = {}
    for j in i.find_all():
      dic_[j.name] = [j.text]
    asos = pd.concat([asos,pd.DataFrame(dic_)],axis=0)
  
  # 한번에 1000건 이상 요청이 불가능한 상황에 대한 조치
  max_ = pd.to_datetime(asos['tm'], format = '%Y-%m-%d').max()
  max_1 = max_ + timedelta(days=1)

  set_startDt_ = str(max_1.date()).replace('-','')

  # 저장된 데이터 프레임과 다음 요청 시 시작될 날짜를 반환
  return asos, set_startDt_

In [2]:
get_data('20110101', '184')

(   stnId stnNm          tm avgTa minTa minTaHrmt maxTa maxTaHrmt mi10MaxRn  \
 0    184    제주  2011-01-01   2.5   0.9       234   3.2         2             
 0    184    제주  2011-01-02   3.5   2.8        47   4.2      1437             
 0    184    제주  2011-01-03   3.5   2.7      1312   4.1      2327             
 0    184    제주  2011-01-04   4.0   2.3       519   4.6      1359             
 0    184    제주  2011-01-05   5.1   4.4         9   6.0      1126             
 ..   ...   ...         ...   ...   ...       ...   ...       ...       ...   
 0    184    제주  2013-09-21  24.9  21.7       615  28.0      1451             
 0    184    제주  2013-09-22  24.6  23.5         1  25.7      1536       5.5   
 0    184    제주  2013-09-23  25.0  22.8       551  27.9      1451       0.3   
 0    184    제주  2013-09-24  25.3  22.2       617  28.1      1349             
 0    184    제주  2013-09-25  21.7  19.9       916  24.6         1       0.5   
 
    mi10MaxRnHrmt  ... avgM05Te avgM10Te avgM15Te 

In [20]:
startDt_ = '20200101'
stnIds_ = '184'

In [21]:
import pandas as pd
asos_ = pd.DataFrame()
while startDt_ != '20220111':
    temp_asos_, startDt_ = get_data(startDt_,stnIds_)
    asos_ = pd.concat([asos_,temp_asos_],axis=0)
    print(startDt_)

20220112


KeyError: 'tm'

In [ ]:
asos_